In [3]:
%load_ext autoreload
import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

import gym
from gym import spaces

tf.compat.v1.enable_v2_behavior()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload 2
from PIL import Image, ImageDraw

coord = np.array([[34, 51, 44, 61], [74, 64, 94, 74], [-5,  4,  5, 24]])

im = Image.new("L", (100, 100))
draw = ImageDraw.Draw(im)

for _, c in enumerate(coord):
    draw.rectangle(tuple(c), fill=256)

# normalize & transform image
# x = np.array(im, dtype=np.float) / 255.0
# x = np.expand_dims(x, axis=-1)  # (H, W, C=1)

np.array(im).max()

255

In [18]:
%autoreload 2
%cd /tf/agents/mysrc

from PIL import Image
from envs.toy import RectEnv, CANVAS_WIDTH

env = RectEnv()
env.reset()

obs, r, _, _ = env.step((1, 1, 4))


# x.shape
# x = (obs['target'] * 256).astype(np.uint8).squeeze(axis=2)
# x.shape

# Image.fromarray(x, mode="L")
# print(r)

/tf/agents/mysrc
[[117  41 127  51]
 [ 88 102 108 112]
 [ 45  96  55 116]]
[[ 0  0 10 10]
 [ 0  0 20 10]
 [ 0  0 10 20]]
[20  0]
[[ 0  0 10 10]
 [ 0  0 20 10]
 [ 0  0 10 20]]
[20  0]
[[ 0  0 10 10]
 [20  0 40 10]
 [ 0  0 10 20]]
[[ 0  0 10 10]
 [20  0 40 10]
 [ 0  0 10 20]]


In [7]:
# np.clip(np.random.rand(2), 0.1, 0.3)
# np.random.uniform(low=0.1, high=0.3, size=(2))
width = 100
rects_wh = np.array([[10, 10], [20, 10], [10, 20]], dtype=np.float) / width
# np.array([np.concatenate([[0, 0], wh]) for wh in rects_wh])
cxy = np.random.rand(3, 2)
bbox = (np.concatenate([cxy-rects_wh/2, cxy+rects_wh/2], axis=1) * width).astype(np.int32)
print((cxy * width).astype(np.int32))
bbox

[[80 37]
 [31 17]
 [13 90]]


array([[ 75,  32,  85,  42],
       [ 21,  12,  41,  22],
       [  8,  80,  18, 100]], dtype=int32)

In [36]:
n_items = 2
# cxy = np.random.rand(n_items, 2)  # (N, 2)
cxy = np.array([[0, 0], [12, 12]])
wh = np.tile([10, 10], (2, 1))
xy0 = cxy - wh/2
xy1 = cxy + wh/2
np.concatenate([xy0, xy1], axis=1)
# np.concatenate([cxy, cxy + np.tile([10, 10], (2, 1))], axis=1)  # (N)
# np.tile([10, 10], (2, 1))
# np.array([[1,1],[2,2]]) + np.array([[3,3],[4,4]])

array([[-5., -5.,  5.,  5.],
       [ 7.,  7., 17., 17.]])

In [23]:
from PIL import Image
w = 128
im = Image.new("L", (w, w))
draw = ImageDraw.Draw(im)

coord = (coord * w).astype(np.int16)
for _, c in enumerate(coord):
    draw.rectangle(tuple(c), fill=255)

# normalize & transform image
x = np.array(im, dtype=np.float32) / 255.0
x = np.expand_dims(x, axis=-1)  # (H, W, C=1)


100000000

In [21]:
%autoreload 2
%cd /tf/agents/mysrc
!python ppo_example.py --root_dir=/tf/agents/log/ppo/gym/HalfCheetah-v2/ --num_parallel_environments=1 --collect_episodes_per_iteration=7 --num_eval_episodes=13 --logtostderr

/tf/agents/mysrc
7 1 13
2020-01-10 18:56:51.898367: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-01-10 18:56:51.898555: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2020-01-10 18:56:51.898632: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (63600c1a1877): /proc/driver/nvidia/version does not exist
2020-01-10 18:56:51.899524: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-01-10 18:56:51.923386: I tensorflow/core/platform/profile_utils/cpu_utils.cc:101] CPU Frequency: 1296000000 Hz
2020-01-10 18:56:51.923705: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5354130 initialized for platform Host (this do

In [48]:
%autoreload 2
%cd /tf/agents/mysrc
from ppo_toy import train

train()

/tf/agents/mysrc


[autoreload of tf_agents.drivers.dynamic_episode_driver failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/tf/agents/tf_agents/drivers/dynamic_episode_driver.py", line 46, in <module>
    class DynamicEpisodeDriver(driver.Driver):
  File "/usr/local/lib/python3.6/dist-packages/gin/config.py", line 1129, in 

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=OrderedDict([('canvas', BoundedTensorSpec(shape=(64, 64, 1), dtype=tf.float32, name='observation/canvas', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32))), ('coord', BoundedTensorSpec(shape=(2, 4), dtype=tf.float32, name='observation/coord', minimum=array(-10., dtype=float32), maximum=array(10., dtype=float32))), ('target', BoundedTensorSpec(shape=(64, 64, 1), dtype=tf.float32, name='observation/target', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)))]))
(BoundedTensorSpec(shape=(), dtype=tf.int64, name='action/tuple_0', minimum=array(0), maximum=array(1)), BoundedTensorSpec(shape=(), dtype=tf.int64, name='action/tuple_1', minimum=array(0), maximum=array(4)), B

InvalidArgumentError:  input must be 4-dimensional[2,1,64,64,1]
	 [[{{node driver_loop/body/_1/ActorDistributionNetwork/EncodingNetwork/sequential_76/conv2d_54/BiasAdd}}]] [Op:__inference_run_917680]

Function call stack:
run


In [19]:
from tf_agents.networks import encoding_network

observation_spec =  {
    'image': tensor_spec.BoundedTensorSpec((16, 16, 3), np.float32, minimum=0, maximum=255),
    'vector': tensor_spec.BoundedTensorSpec((5,), np.float32, minimum=-100, maximum=100)}
preprocessing_layers = {
    'image': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
                                        tf.keras.layers.Flatten()]),
    'vector': tf.keras.layers.Dense(5)}
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)


encoder = encoding_network.EncodingNetwork(
        input_tensor_spec=observation_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
#         conv_layer_params=conv_layer_params,
#         fc_layer_params=fc_layer_params,
#         dropout_layer_params=dropout_layer_params,
#         activation_fn=activation_fn,
#         kernel_initializer=kernel_initializer,
#         batch_squash=batch_squash,
#         dtype=dtype
)

state, network_state = encoder({
    "image":tf.random.uniform((16,16,3)),
    "vector": tf.random.uniform((5,))
})
state, network_state

(<tf.Tensor: shape=(1357,), dtype=float32, numpy=
 array([ 0.09738518,  0.425965  , -0.06698334, ...,  0.19842806,
         0.7509032 , -0.73973745], dtype=float32)>, ())